Шаблон для запуска парсинга: \
python parsing.py 2 True housedorf urls_housedorf.txt example.xlsx result_housedorf.xlsx 
python parsing.py 2 False falmec urls_falmec.txt result_housedorf.xlsx result.xlsx

python generate_syn_report.py housedorf syn_report_housedorf.xlsx
python generate_syn_report.py falmec syn_report_falmec.xlsx

python synonyms_dict_update.py syn_report_housedorf.xlsx
python synonyms_dict_update.py syn_report_falmec.xlsx

python parsing.py 2 True housedorf urls_housedorf.txt example.xlsx result_housedorf.xlsx 

python parsing.py 2 False falmec urls_falmec.txt result_housedorf.xlsx result.xlsx

python compare.py falmec housedorf

python parsing.py 24 False smeg urls_smeg.txt result.xlsx result.xlsx 
python compare.py smeg housedorf

python parsing.py 2 True housedorf urls_housedorf.txt articles.xlsx result_housedorf.xlsx 
python parsing.py 2 False franke_dealer urls_franke-dealer.txt result_gerdamix.xlsx result_franke.xlsx
python compare.py blanco housedorf

# Программа parsing.py

In [1]:
import urllib3
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Alignment
import re
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import argparse
import os
from ruwordnet import RuWordNet
import pymorphy2
from itertools import product
from pathlib import Path
import requests

synonyms_path='synonyms.txt'
all_characteristics_path='all_characteristics.xlsx'

In [2]:
full_str = ''
full_str = '; '.join([full_str, 'new_str'])
full_str

'; new_str'

In [5]:
# ------------------------------------------------------------------------Спарсить данные------------------------------------------------------------------------

# Функции для парсинга
def parse_korting_page(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
    tabs_lists = soup.find_all('ul', class_='tabs-settings__list')
    data = {}

    for ul in tabs_lists:
        for li in ul.find_all('li'):
            text = li.get_text(strip=True, separator="; ")
            split_text = text.split(":;", 1)
            if len(split_text) == 2:
                key, value = split_text
                data[key.strip()] = value.strip()
            else:
                data[split_text[0].strip()] = ""

    return data

def parse_dedietrich_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='characteristics__row'):
        name_span = row.find('span', class_='characteristics__name')
        value_span = row.find('span', class_='characteristics__property')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_granfest_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='item__desc-char'):
        name_span = row.find('div', class_='item__desc-char-name')
        value_span = row.find('div', class_='item__desc-char-value')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_topzero_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    row = soup.find('div', class_='item-desc-size')
    if row:
        key, value = ''.join(row.find_all(text=True, recursive=True)).split(':')
        if key and value:
            key = key.strip()
            value = value.strip().replace('\n', '; ')
            data[key] = value
    items = soup.find('div', class_='item-desc-other')

    for row in items.find_all('div', recursive=False):
        full_text = ''.join(row.find_all(text=True, recursive=True))
        if ':' in full_text:
            key, value = full_text.split(':')
            if key and value:
                key = key.strip()
                value = value.strip().replace('\n', '; ')
                data[key] = value
        else:
            if not 'Особенности' in data.keys():
                data['Особенности'] = full_text.replace('Особенности', '').strip().replace('\n', '; ')

    return data

def parse_mypremial_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    info_div = soup.find('div', class_='the-item__info')

    # Проходим по всем div с классом characteristics__row
    for row in info_div.find_all('p'):
        name_span = row.find('span', class_='info__title')
        value_span = row.find('span', class_='info__value')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip().split(':')[0]
                value = value.strip()
                data[key] = value

    return data

def parse_gerdamix_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    info_div = soup.find('div', class_='col-lg-5 col-md-5 col-sm-5 col-xs-12 all_opisanie plusi')

    full_str = ''
    for row in info_div.find_all('li', recursive=True):
        new_str = row.find(text=True, recursive=False).strip()
        if full_str != '':
            full_str = '; '.join([full_str, new_str])
        else:
            full_str = new_str
    
    data['Особенности'] = full_str

    return data

def parse_ukinox_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    info_div = soup.find('div', class_='single-har')

    # Проходим по всем div с классом characteristics__row
    for row in info_div.find_all('li'):
        name_span, value_span = row.find_all('p')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip().split(':')[0]
                value = value.strip()
                data[key] = value

    return data

def parse_rivelato_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    info_div = soup.find('ul', class_='params-list')

    # Проходим по всем div с классом characteristics__row
    for row in info_div.find_all('li'):
        name_span, value_span = row.find_all('span')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=True)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_aquaphor_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('li', class_='equipment-row'):
        name_span = row.find('span', class_='equipment-row__name')
        value_span = row.find('span', class_='equipment-row__value')
        
        if name_span and value_span:
            key = ''.join(name_span.find_all(text=True, recursive=True)).strip()
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data


def parse_makmart_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='prop'):
        name_span = row.find('div', class_='name')
        value_span = row.find('div', class_='value')
        
        if name_span and value_span:
            key = name_span.get_text(strip=True)
            value = value_span.get_text(strip=True)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_longran_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    row = soup.find('div', class_='catalog-single-har-right')

    if row:
        for li in row.find_all('li'):
            name_span, value_span = li.find_all('p')
            
            if name_span and value_span:
                key = name_span.find(text=True, recursive=False).split(":")[0]
                value = value_span.find(text=True, recursive=False)
                if key and value:
                    key = key.strip()
                    value = value.strip()
                    data[key] = value
    
    for row in soup.find_all('div', class_='new-product-specifications__item'):
        name_span, value_span = row.find_all('div')
        if name_span and value_span:
                key = name_span.find(text=True, recursive=False).split(":")[0]
                value = value_span.find(text=True, recursive=False)
                if key and value:
                    key = key.strip()
                    value = value.strip()
                    data[key] = value

    return data

def parse_fashun_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    rough = soup.find('div', class_='js-store-prod-all-text')

    # Разделяем содержимое по тегам <br>
    if rough:
        parts = []
        current = []
        for elem in rough.children:
            if getattr(elem, 'name', None) == 'br':
                text = ''.join(current).strip()
                if text:
                    parts.append(text)
                current = []
            else:
                current.append(str(elem))
        # Добавляем последний кусок, если он есть
        if current:
            text = ''.join(current).strip()
            if text:
                parts.append(text)

        str_parts = [BeautifulSoup(t, 'html.parser').get_text(strip=True) for t in parts]
        for part in str_parts:
            if ':' in part:
                key, value = part.split(':', 1)
                data[key.strip()] = value.strip()
            if '=' in part:
                key, value = part.split('=', 1)
                data[key.strip()] = value.strip()

    charcs = soup.find('div', class_='js-store-prod-all-charcs')
    if charcs:
        for row in charcs.find_all('p'):
            key, value = row.find(text=True, recursive=False).split(':', 1)
            data[key.strip()] = value.strip()

    return data

def parse_blanco_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    rough = soup.find('table', class_='product-attributes')

    # Проходим по всем div с классом characteristics__row
    for row in rough.find_all('tr'):
        name_th = row.find('th')
        value_th = row.find('td')
        
        if name_th and value_th:
            key = name_th.find(text=True, recursive=False)
            value = value_th.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_geizer_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    rough = soup.find('table', class_='attribute')

    # Проходим по всем div с классом characteristics__row
    for tbody in rough.find_all('tbody'):
        for tr in tbody.find_all('tr'):
            tds = tr.find_all('td')
            if len(tds) == 2:
                name_td, value_td = tds
                key = name_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                if key and value:
                    data[key] = value
    return data


def parse_vzug_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}
    names = soup.find_all('td', class_='cell_name')
    values = soup.find_all('td', class_='cell_value')

    # Проходим по всем div с классом characteristics__row
    for name, value in zip(names, values):
        name_span = name.find('span')
        value_span = value.find('span')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_asco_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='accordeon__item'):
        name_span = row.find('span', class_='accordeon__item-title')
        value_span = row.find('p', class_='accordeon__item-text')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_kuppersbush_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='wdu_propsorter'):
        for item in row.find_all('tr'):
            tds = item.find_all('td')

            if tds:
                name_td, value_td = tds
                key = name_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                if key and value:
                    data[key] = value

    return data

def parse_shaublorenz_shop_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('tr', class_='tablerow'):
        text = row.find_all(text=True, recursive=True)
        
        if text:
            key = text[0]
            value = text[2]
            data[key] = value

    return data

def parse_shaublorenz_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='attr-group-box'):
        name_div = row.find('div', class_='attr-name')
        value_div = row.find('div', class_='attr-value')
        
        if name_div and value_div:
            key = name_div.find(text=True, recursive=False)
            value = value_div.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_evelux_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='product__content-specs-line'):
        name_span = row.find('div', class_='product__content-specs-title')
        value_span = row.find('div', class_='product__content-specs-subtitle')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False).split(':')[0]
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_graude_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for name_td, value_td in zip(soup.find_all('td', class_='cell_name'), soup.find_all('td', class_='cell_value')):
        key = name_td.find(text=True, recursive=True)
        value = value_td.find(text=True, recursive=True)
        if key and value:
            key = key.strip().split(':')[0]
            value = value.strip()
            data[key] = value

    return data

def parse_franke_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='cmp-product-information-table__section-list__item'):
        name_span = row.find('span', class_='cmp-product-information-table__section-list__item__name')
        value_span = row.find('span', class_='cmp-product-information-table__section-list__item__value')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_smeg_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('tr', class_='s-feature-column'):
        name_td = row.find('td', class_='name')
        value_td = row.find('td', class_='value')
        
        if name_td and value_td:
            key = name_td.find(text=True, recursive=True)
            value = value_td.find(text=True, recursive=True)
            if key and value and "фид" not in key:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_history_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('table', class_='alignleft'):
        for tr in row.find_all('tr'):
            tds = tr.find_all('td')
            
            if len(tds) == 2:
                name_td, value_td = tds
                key = name_td.find(text=True, recursive=False)
                value = value_td.find(text=True, recursive=False)
                if key and value:
                    key = key.strip()
                    value = value.strip()
                    data[key] = value

    return data

def parse_elica_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='ty-product-feature'):
        name_span = row.find('span', class_='ty-product-feature__label')
        value_div = row.find('div', class_='ty-product-feature__value')
        
        if name_span and value_div:
            key = name_span.find(text=True, recursive=False)
            value = value_div.find(text=True, recursive=False)
            if key and value:
                key = key.strip().rstrip('.,;!?—:').split('\n')[0]
                value = value.strip().rstrip('.,;!?—:').split('\n')[0]
                data[key] = value
            elif key:
                checkbox = value_div.find('span', class_='ty-compare-checkbox')
                title = checkbox.get('title')
                if title == 'Y':
                    value = "Да"
                    data[key.rstrip('.,;!?—:').split('\n')[0]] = value

    return data

def parse_franke_dealer_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('tr'):
        name_td = row.find('td', class_='name')
        value_td = row.find('td', class_='value')
        
        if name_td and value_td:
            key = name_td.find(text=True, recursive=False)
            value = value_td.find(text=True, recursive=False)
            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def parse_konigin_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # 1. Извлекаем div.column и все p внутри
    column_div = soup.find('div', class_='column')
    if column_div:
        p_tags = column_div.find_all('p')
        # Игнорируем первые 2 и последние 3
        middle_p_tags = p_tags[2:-3]

        # Соединяем текст через "; "
        additional_info = "; ".join(p.get_text(strip=True).rstrip('.,;!?—:') for p in middle_p_tags)
        data['ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ'] = additional_info

        # Обрабатываем последние два тега <p>
        for p in p_tags[-2:]:
            text = p.get_text(strip=True)
            # Делим по точкам
            for part in text.split('.'):
                if ':' in part:
                    key, value = part.split(':', 1)
                    data[key.strip()] = value.strip()

    # 2. Обрабатываем блоки div.card-spec__item
    spec_items = soup.find_all('div', class_='card-spec__item')
    for item in spec_items:
        title_div = item.find('div', class_='card-spec__item-title')
        info_div = item.find('div', class_='card-spec__item-info')
        if title_div and info_div:
            key = title_div.get_text(strip=True)
            value = info_div.get_text(strip=True).rstrip('.,;!?—:')
            data[key] = value

    return data

def parse_falmec_page(html_code: str) -> dict:
    soup = BeautifulSoup(html_code, 'html.parser')
    data = {}

    # Проходим по всем div с классом characteristics__row
    for row in soup.find_all('div', class_='characteristics__row'):
        name_span = row.find('span', class_='characteristics__name')
        value_span = row.find('span', class_='characteristics__property')
        
        if name_span and value_span:
            key = name_span.find(text=True, recursive=False)
            value = value_span.find(text=True, recursive=False)

            # Если нет простого текста, ищем <ul> и собираем <li>
            if (value == '' or not value.strip()) and value_span.find('ul'):
                li_items = value_span.find_all('li')
                value = '; '.join(li.get_text(strip=True) for li in li_items)

            if key and value:
                key = key.strip()
                value = value.strip()
                data[key] = value

    return data

def extract_first_visible_text(tag):
    for desc in tag.descendants:
        if isinstance(desc, str):  # Это NavigableString
            text = desc.strip()
            if text:
                return text
    return None

def clean_value_div(value_div):
    # 1. Удалить все <span>
    for span in value_div.find_all("span"):
        span.decompose()

    # 2. Разделить по <br> — создаём список на основе HTML с разделителем
    parts = str(value_div).split('<br')

    values = []

    for part_html in parts:
        # Восстанавливаем HTML-тег <br>, если он был отрезан
        if not part_html.startswith('>'):
            part_html = '<br' + part_html

        part_soup = BeautifulSoup(part_html, 'html.parser')

        # 3. Найти первый видимый текст
        for desc in part_soup.descendants:
            if isinstance(desc, NavigableString):
                text = desc.strip()
                if text:
                    values.append(text)
                    break  # только первое вхождение

    # 4. Склеить с разделителем "; "
    return "; ".join(values)

def parse_hausedorf_page(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
    fields = soup.find_all('div', class_='detail-properties__field')
    data = {}

    for field in fields:
        name_div = field.find('div', class_='detail-properties__name')
        value_div = field.find('div', class_='detail-properties__value')

        if name_div and value_div:
            key = extract_first_visible_text(name_div)
            value = clean_value_div(value_div)

            if key and value:
                data[re.sub(r'\s+', ' ', key).strip()] = value.replace(">\n", "")

    return data


def create_src(file_path, parser_func):
    """
    Универсальный загрузчик таблицы характеристик с разных сайтов.

    :param file_path: путь к файлу с URL (один URL на строку)
    :param parser_func: функция, которая получает HTML-код и возвращает словарь {ключ: значение}
    :return: DataFrame с объединёнными результатами
    """
    df_all = pd.DataFrame()

    with open(file_path, 'r', encoding='utf-8') as f:
        urls = [line.strip() for line in f]

    n_rows = 0
    for url in tqdm(urls):
        if url:
            try:
                response = requests.get(url)
                response.encoding = 'utf-8'
                # response.encoding = response.apparent_encoding
                html_code = response.text
                data = parser_func(html_code) 

                if not isinstance(data, dict):
                    raise ValueError("parser_func должна возвращать словарь!")

                row_df = pd.DataFrame([data])
                df_all = pd.concat([df_all, row_df], ignore_index=True)

                if len(df_all) == 1:
                    empty_rows = pd.DataFrame(np.nan, index=range(n_rows), columns=df_all.columns)
                    df_all = pd.concat([empty_rows, df_all], ignore_index=True)

            except Exception as e:
                print(f"Ошибка при обработке {url}: {e}")
        else:
            if len(df_all.columns) > 0:                
                df_all.loc[len(df_all)] = None
            else:
                n_rows += 1

    return df_all.where(pd.notnull(df_all), None)

# ---------------------------------------------Записать и вернуть дополненный названиями номенклатуры DataFrame---------------------------------------------

def write_dest(ref_file_path, result_file_path, df_src, start_row_index):
    # Путь к файлу Excel
    wb = load_workbook(ref_file_path)
    ws = wb.active  # или wb['SheetName']

    # Поля файла-приёмника
    row_header = [cell.value for cell in ws[1]]

    # Сопоставление колонок
    src_cols_lower = {col.lower(): col for col in df_src.columns}
    ws_cols_lower = {i: str(header).strip().lower() if header else "" for i, header in enumerate(row_header)}
    matched_columns = []
    common_cols = set()
    nomenclature_col_idx = None

    for col_idx, header_lower in ws_cols_lower.items():
        if header_lower == "номенклатура":
            nomenclature_col_idx = col_idx
        if header_lower in src_cols_lower:
            matched_columns.append((col_idx, src_cols_lower[header_lower]))
            common_cols.add(src_cols_lower[header_lower])

    if nomenclature_col_idx is None:
        raise ValueError("Колонка 'Номенклатура' не найдена в файле-приёмнике")

    # Считываем значения "Номенклатура"
    nomenclature_values = []
    for i in range(len(df_src)):
        cell_value = ws.cell(row=start_row_index + i, column=nomenclature_col_idx + 1).value
        nomenclature_values.append(cell_value)

    # Запись данных
    for i, (_, row_src) in enumerate(df_src.iterrows()):
        for col_idx, src_col in matched_columns:
            cell = ws.cell(row=start_row_index + i, column=col_idx + 1)
            if cell.value in [None, ""]:
                cell.value = row_src[src_col]

    # Сохранение
    wb.save(result_file_path)

    # Подготовка выходного DataFrame
    result_df = df_src[list(common_cols)].copy()
    result_df.insert(0, "Номенклатура", pd.Series(nomenclature_values))

    return result_df, common_cols

# -------------------------------------------Сохранить незаписанные данные в дополнительные колонки или отдельный файл-------------------------------------------

def append_dataframe_to_excel(df: pd.DataFrame, file_path: str, result_path: str, start_row: int):
    # Проверка, существует ли файл
    if os.path.exists(file_path):
        wb = load_workbook(file_path)
    else:
        # Если файла нет, создаем новый
        wb = Workbook()
    
    ws = wb.active

    # Найдём первую пустую ячейку в первой строке
    col_index = 1
    while ws.cell(row=1, column=col_index).value is not None:
        col_index += 1

    # Записываем названия колонок DataFrame в первую строку, начиная с найденной колонки
    for i, col_name in enumerate(df.columns):
        cell = ws.cell(row=1, column=col_index + i, value=col_name)
        if len(col_name.split('_')) > 1:
            cell.fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
            

    # Автонастройка ширины столбцов по первой строке
    for col_idx, cell in enumerate(ws[1], start=col_index):
        max_length = len(str(cell.value)) if cell.value else 0
        col_letter = cell.column_letter
        ws.column_dimensions[col_letter].width = max_length + 2  # +2 для отступа

    # Применение стилей и переносов
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center')  # включаем перенос текста

    # Записываем данные DataFrame начиная со start_row
    for row_offset, row in enumerate(dataframe_to_rows(df, index=False, header=False)):
        for i, value in enumerate(row):
            ws.cell(row=start_row + row_offset, column=col_index + i, value=value)

    # Сохраняем файл
    wb.save(result_path)

def save_missing(df1, filepath):

    # Создаём ExcelWriter
    with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
        row = 0

        # Запись df1
        df1.to_excel(writer, index=False, startrow=row)
        row += len(df1) + 2  # +1 за заголовок, +1 за пустую строку

        # Автоматическая установка ширины колонок
        worksheet = writer.sheets['Sheet1']
        for column_cells in worksheet.columns:
            max_length = 0
            column = column_cells[0].column
            for cell in column_cells:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            adjusted_width = max_length + 2
            worksheet.column_dimensions[get_column_letter(column)].width = adjusted_width

In [6]:
# Протестировать функцию парсинга
def test_parser(url, parser_func):
    response = requests.get(url)
    response.encoding = 'utf-8'
    # response.encoding = response.apparent_encoding
    html_code = response.text
    data = parser_func(html_code)
    return data

data = test_parser('https://gerdamix.ru/dorado/', parse_gerdamix_page)
print(len(data))  # Должно вернуть количество характеристик, например 10
data # Выводим результат парсинга

1


{'Особенности': 'латунь; выход для питьевой воды; высокий поворотный излив; однорычажный; керамический картридж 35 мм; гибкая подводка 400 мм; гарантия 3 года'}

In [17]:
# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def list_synonyms_comparison(list1, list2):
    return [are_synonyms(word1, word2) for word1, word2 in zip(list1, list2)]

In [15]:
# Протестировать подбор синонимов
list1 = ['Машина', 'Счастливый', 'Работать']
list2 = ['автомобиля', 'счастливый', 'работник']
list_synonyms_comparison(list1, list2)

[True, True, False]

In [25]:
# -----------------------------Добавить функцию для обеспечения правильного дописывания данных основываясь на утверждённых синонимах-----------------------------

def parse_custom_dict_line(line):
    """
    Разбирает строку из словаря: <характеристика>: <синоним1>; <синоним2>; ...| <антисиноним1>, <антисиноним2>, ...
    """
    base, *rest = line.strip().split(':')
    if not rest:
        return base.strip(), set(), set()
    syn_ant = rest[0].split('|')
    synonyms = set(map(str.strip, syn_ant[0].split(';'))) if syn_ant[0] else set()
    antisynonyms = set(map(str.strip, syn_ant[1].split(','))) if len(syn_ant) > 1 else set()
    return base.strip(), synonyms, antisynonyms

def load_existing_synonyms(file_path):
    syn_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            base, synonyms, antisynonyms = parse_custom_dict_line(line)
            syn_dict[base] = {'synonyms': synonyms, 'antisynonyms': antisynonyms}
    return syn_dict

def rename_columns_with_syn_dict(df, syn_dict_path, all_1c_chars_path):
    # Загрузка словаря
    synonyms_dict = load_existing_synonyms(syn_dict_path)
    all_chars = pd.read_excel(all_1c_chars_path, header=None).iloc[0].astype(str).tolist()
    all_chars_lower = [char.lower() for char in all_chars]  # Приводим к нижнему регистру для сравнения

    # Удаление дублирующихся колонок по имени
    df = df.loc[:, ~df.columns.duplicated()]

    # Поиск синонимичных имен колонок через словарь
    df_expanded = df.copy()
    unsyn_set = set()
    for col in df.columns:
        col_splitted = col.split(':')[0]
        # Если колонка уже есть в 1С - пропускаем её
        if col_splitted.lower() in all_chars_lower:
            df_expanded[col_splitted.upper()] = df[col]
            df_expanded.drop(columns=[col], inplace=True)
            continue

        is_syn = False
        for synonym_key, syn_data in synonyms_dict.items():
            syn_set = syn_data.get("synonyms", set())
            if col_splitted in syn_set:
                # Добавляем колонку с именем synonym_key, если она ещё не существует
                if synonym_key not in df_expanded.columns:
                    df_expanded[synonym_key] = df[col]
                    is_syn = True
        if is_syn:
            # Если колонка была переименована, удаляем оригинальную и не ищем полусинонимы
            df_expanded.drop(columns=[col], inplace=True)
        else:
            is_half_syn = False
            for synonym_key, syn_data in synonyms_dict.items():
                syn_set = syn_data.get("synonyms", set())
                if col_splitted in set(map(lambda x: x.split("*")[1] if len(x.split("*")) > 1 else x, syn_set)):
                    # Добавляем колонку с именем synonym_key, если она ещё не существует
                    if synonym_key not in df_expanded.columns:
                        df_expanded[f"{synonym_key}_{col_splitted}"] = df[col]
                        is_half_syn = True

            if is_half_syn:
                df_expanded.drop(columns=[col], inplace=True)
            else:
                is_antisyn = False
                for synonym_key, syn_data in synonyms_dict.items():
                    antisyn_set = syn_data.get("antisynonyms", set())
                    if col_splitted in antisyn_set:
                        is_antisyn = True
                        break

                if not is_antisyn:
                    unsyn_set.add(col_splitted)

    return df_expanded, unsyn_set

# Варианты запуска парсинга

In [ ]:
# -----------------------------------------------------------Запуск парсинга и сохранение результатов-----------------------------------------------------------

# Протестировать запись одной строки с двух сайтов
from argparse import Namespace

# housedorf
args = Namespace(start_row=33, append=True, site='housedorf', urls_source='input_example/url_housedorf.txt', input_path='input_example/example.xlsx', output_path='result_housedorf.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    missingdf.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(missingdf, f'missing_{args.site}.xlsx')

# korting
args = Namespace(start_row=34, append=False, site='korting', urls_source='input_example/url_korting.txt', input_path='result_housedorf.xlsx', output_path='result.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    missingdf.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(missingdf, f'missing_{args.site}.xlsx')

In [ ]:
# Актуальная версия запуска
from argparse import Namespace

# housedorf
args = Namespace(start_row=7, append=False, site='falmec', urls_source='url_falmec.txt', input_path='result_housedorf.xlsx', output_path='result.xlsx')
if not args.output_path:
    raise ValueError("there's not enough arguments")

# Проверка, не повреждены ли входные файлы
wb = load_workbook(args.input_path)

if args.site == 'korting':
    df_src = create_src(args.urls_source, parse_korting_page)
elif args.site == 'housedorf':
    df_src = create_src(args.urls_source, parse_hausedorf_page)
elif args.site == 'dedietrich':
    df_src = create_src(args.urls_source, parse_dedietrich_page)
elif args.site == 'falmec':
    df_src = create_src(args.urls_source, parse_falmec_page)
elif args.site == 'vzug':
    df_src = create_src(args.urls_source, parse_vzug_page)
elif args.site == 'asco':
    df_src = create_src(args.urls_source, parse_asco_page)
elif args.site == 'kuppersbush':
    df_src = create_src(args.urls_source, parse_kuppersbush_page)
elif args.site == 'konigin':
    df_src = create_src(args.urls_source, parse_konigin_page)
elif args.site == 'evelux':
    df_src = create_src(args.urls_source, parse_evelux_page)
elif args.site == 'franke':
    df_src = create_src(args.urls_source, parse_franke_page)
elif args.site == 'franke_dealer':
    df_src = create_src(args.urls_source, parse_franke_dealer_page)
elif args.site == 'elica':
    df_src = create_src(args.urls_source, parse_elica_page)
elif args.site == 'smeg':
    df_src = create_src(args.urls_source, parse_smeg_page)
elif args.site == 'shaublorenz':
    df_src = create_src(args.urls_source, parse_shaublorenz_page)
elif args.site == 'shaublorenz_shop':
    df_src = create_src(args.urls_source, parse_shaublorenz_shop_page)
elif args.site == 'graude':
    df_src = create_src(args.urls_source, parse_graude_page)
elif args.site == 'history':
    df_src = create_src(args.urls_source, parse_history_page)
elif args.site == 'blanco':
    df_src = create_src(args.urls_source, parse_blanco_page)
elif args.site == 'fashun':
    df_src = create_src(args.urls_source, parse_fashun_page)
elif args.site == 'geizer':
    df_src = create_src(args.urls_source, parse_geizer_page)
elif args.site == 'longran':
    df_src = create_src(args.urls_source, parse_longran_page)
elif args.site == 'makmart':
    df_src = create_src(args.urls_source, parse_makmart_page)
elif args.site == 'aquaphor':
    df_src = create_src(args.urls_source, parse_aquaphor_page)
elif args.site == 'mypremial':
    df_src = create_src(args.urls_source, parse_mypremial_page)
elif args.site == 'rivelato':
    df_src = create_src(args.urls_source, parse_rivelato_page)
elif args.site == 'topzero':
    df_src = create_src(args.urls_source, parse_topzero_page)
elif args.site == 'ukinox':
    df_src = create_src(args.urls_source, parse_ukinox_page)
elif args.site == 'granfest':
    df_src = create_src(args.urls_source, parse_granfest_page)
elif args.site == 'gerdamix':
    df_src = create_src(args.urls_source, parse_gerdamix_page)
else:
    raise ValueError("There're no parse function for this site")

df_src, unsyn_set = rename_columns_with_syn_dict(df_src, synonyms_path, all_characteristics_path) # Для обеспечения правильного дописывания данных и для корректного входа к функции, 
                                                                        # генерирующей отчёт
resultdf, com_cols = write_dest(args.input_path, args.output_path, df_src, args.start_row)
resultdf.to_parquet(f"{args.site}_auxiliary.parquet")

com_cols = resultdf.columns.intersection(df_src.columns)
missingdf = df_src.drop(columns=com_cols).copy()
print(unsyn_set)
with open(f'unaccepted_syn_{args.site}.txt', 'w', encoding='utf-8') as f:
    f.write('; '.join(map(str, unsyn_set)))

if args.append:
    append_dataframe_to_excel(missingdf, args.output_path, args.output_path, args.start_row)
else:
    # Условие: имена столбцов, у которых есть хотя бы один символ "_"
    columns_with_underscore = [col for col in missingdf.columns if len(col.split("_")) > 1]
    columns_without_underscore = [col for col in missingdf.columns if len(col.split("_")) <= 1]

    # Делим missingdf на два
    df_with_underscore = missingdf[columns_with_underscore]
    df_without_underscore = missingdf[columns_without_underscore]
    df_without_underscore.insert(0, 'Номенклатура', resultdf['Номенклатура'].copy())
    save_missing(df_without_underscore, f'missing_{args.site}.xlsx')
    append_dataframe_to_excel(df_with_underscore, args.output_path, args.output_path, args.start_row)

100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


set()


# Программа generate_syn_report.py

In [1]:
from ruwordnet import RuWordNet
import pymorphy2
import pandas as pd
from tqdm import tqdm
import re
from itertools import product

synonyms_path='synonyms.txt'
all_characteristics_path='all_characteristics.xlsx'

In [2]:
# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def list_synonyms_comparison(list1, list2):
    return [are_synonyms(word1, word2) for word1, word2 in zip(list1, list2)]

In [3]:
# -----------------------------------Добавить функцию для создания отчёта о сопоставлении колонок тем, что уже существуют в 1С-----------------------------------

def parse_custom_dict_line(line):
    """
    Разбирает строку из словаря: <характеристика>: <синоним1>; <синоним2>; ...| <антисиноним1>, <антисиноним2>, ...
    """
    base, *rest = line.strip().split(':')
    if not rest:
        return base.strip(), set(), set()
    syn_ant = rest[0].split('|')
    synonyms = set(map(str.strip, syn_ant[0].split(';'))) if syn_ant[0] else set()
    antonyms = set(map(str.strip, syn_ant[1].split(','))) if len(syn_ant) > 1 else set()
    return base.strip(), synonyms, antonyms

def load_existing_synonyms(file_path):
    syn_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            base, synonyms, antonyms = parse_custom_dict_line(line)
            syn_dict[base] = {'synonyms': synonyms, 'antonyms': antonyms}
    return syn_dict

def tokenize(text):
    return set(re.findall(r'\w+', text.lower()))

def are_words_possibly_synonyms(words1, words2, are_synonims_func):
    pairs = product(words1, words2)
    for w1, w2 in pairs:
        if w1 == w2:
            return True
        if are_synonims_func(w1, w2):
            return True
    return False


def generate_synonym_report(existing_dict_path, all_1c_chars_path, new_chars, are_synonims_func, output_excel_path):
    custom_dict = load_existing_synonyms(existing_dict_path)
    all_chars = pd.read_excel(all_1c_chars_path, header=None).iloc[0].astype(str).tolist()
    result_rows = []

    for c1 in tqdm(all_chars):
        for c2 in new_chars:
            if c1 == c2:
                continue
            tokens1 = tokenize(c1)
            tokens2 = tokenize(c2)
            if are_words_possibly_synonyms(tokens1, tokens2, are_synonims_func):
                result_rows.append((c1, c2, None))  # None для ручной отметки

    df_result = pd.DataFrame(result_rows, columns=["base_char", "compared_char", "label"])
    df_result.to_excel(output_excel_path, index=False)

In [ ]:
# -----------------------------------------------------------------------Запуск генерации-----------------------------------------------------------------------

from argparse import Namespace

args = Namespace(site='housedorf', synonyms_report_path='syn_report_housedorf.xlsx')
if not args.synonyms_report_path:
    raise ValueError("there's not enough arguments")

with open(f'unaccepted_syn_{args.site}.txt', 'r', encoding='utf-8') as f:
    data = f.read()

unsyn_list = list(map(str, data.strip().split('; ')))

generate_synonym_report(
    synonyms_path,
    all_characteristics_path,
    unsyn_list,
    are_synonyms,
    args.synonyms_report_path
)

# Программа synonyms_dict_update.py

In [2]:
import pandas as pd
from collections import defaultdict
import os
import argparse
synonyms_path='synonyms.txt'

In [11]:
# --------------------------------------------------------------Обновить словарь синонимов из Excel--------------------------------------------------------------
def load_synonym_dict(dict_path):
    syn_dict = defaultdict(lambda: {"synonyms": set(), "antisynonyms": set()})
    if not os.path.exists(dict_path):
        return syn_dict
    
    with open(dict_path, "r", encoding="utf-8") as f:
        for line in f:
            if ":" not in line:
                continue
            key, rest = line.strip().split(":", 1)
            syn_part, *anti_part = rest.strip().split("|")
            syns = set(map(str.strip, syn_part.strip().split(";"))) if syn_part.strip() else set()
            antis = set(map(str.strip, anti_part[0].strip().split(";"))) if anti_part and anti_part[0].strip() else set()
            syn_dict[key.strip()]["synonyms"].update(syns)
            syn_dict[key.strip()]["antisynonyms"].update(antis)
    return syn_dict


def save_synonym_dict(syn_dict, dict_path):
    with open(dict_path, "w", encoding="utf-8") as f:
        for key in sorted(syn_dict.keys()):
            syns = "; ".join(sorted(syn_dict[key]["synonyms"]))
            antis = "; ".join(sorted(syn_dict[key]["antisynonyms"]))
            line = f"{key}: {syns} | {antis}\n"
            f.write(line)


def update_synonym_dict_from_excel(excel_path, dict_path):
    df = pd.read_excel(excel_path)
    if not {"base_char", "compared_char", "label"}.issubset(df.columns):
        raise ValueError("Excel должен содержать столбцы: base_char, compared_char, label")
    
    syn_dict = load_synonym_dict(dict_path)

    for _, row in df.iterrows():
        base = row["base_char"].strip().split(":")[0]
        comp = row["compared_char"].strip().split(":")[0]
        label = row["label"]

        if label == 1:
            syn_dict[base]["synonyms"].add(comp)
        elif label == 0.5:
            syn_dict[base]["synonyms"].add("*"+comp)
        elif label == 0 or pd.isna(label):
            syn_dict[base]["antisynonyms"].add(comp)
        else:
            continue  # Пропустить некорректные значения

    # Удалить пересекающиеся значения
    for base in syn_dict:
        overlap1 = syn_dict[base]["synonyms"] & syn_dict[base]["antisynonyms"]
        overlap2 = set(map(lambda x: x.split("*")[1] if len(x.split("*")) > 1 else x, syn_dict[base]["synonyms"])) & syn_dict[base]["antisynonyms"]
        syn_dict[base]["antisynonyms"] -= overlap1
        syn_dict[base]["antisynonyms"] -= overlap2

    save_synonym_dict(syn_dict, dict_path)
    print(f"Обновлённый словарь сохранён в {dict_path}")


In [ ]:
# Запустить обновление файла-словаря

from argparse import Namespace

# housedorf
args = Namespace(report_path='syn_report_housedorf.xlsx')
if not args.report_path:
    raise ValueError("there's not enough arguments")

update_synonym_dict_from_excel(args.report_path, synonyms_path)

Обновлённый словарь сохранён в synonyms.txt


In [ ]:
# Протестировать обновление файла-словаря
syn_dict = load_synonym_dict('synonyms.txt')
for base in syn_dict:
    if len(syn_dict[base]['synonyms']) > 0:
        print(f"{base}: {syn_dict[base]['synonyms']}, {syn_dict[base]['antisynonyms']}")

# Программа compare.py

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import argparse
from itertools import product
from ruwordnet import RuWordNet
import pymorphy2
import re

In [18]:
# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def tokenize(text):
    return set(re.findall(r'\w+', text.lower()))

def are_words_possibly_synonyms(words1, words2, are_synonims_func):
    pairs = product(words1, words2)
    for w1, w2 in pairs:
        if w1 == w2:
            return True
        if are_synonims_func(w1, w2):
            return True
    return False

In [22]:
# -------------------------------------------------------------------Сравнить записанные данные-------------------------------------------------------------------

def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame, name1: str = 'df1', name2: str = 'df2') -> pd.DataFrame:
    """
    Сравнивает два DataFrame по общим столбцам, включая "Номенклатура" как ключ.
    Добавляет префиксы к столбцам (кроме "Номенклатура") и формирует колонку diff_columns.
    """
    if 'Номенклатура' not in df1.columns or 'Номенклатура' not in df2.columns:
        raise ValueError("Оба DataFrame должны содержать колонку 'Номенклатура'")

    # Определим общие характеристики (кроме "Номенклатура")
    common_columns = df1.columns.intersection(df2.columns).difference(['Номенклатура'])

    # Сузим входные DataFrame'ы до нужных колонок
    df1_reduced = df1[['Номенклатура'] + list(common_columns)].copy()
    df2_reduced = df2[['Номенклатура'] + list(common_columns)].copy()

    # Переименуем характеристики с префиксами, "Номенклатура" оставим без изменений
    df1_renamed = df1_reduced.rename(columns={col: f'{name1}_{col}' for col in common_columns})
    df2_renamed = df2_reduced.rename(columns={col: f'{name2}_{col}' for col in common_columns})

    # Объединение по "Номенклатура"
    df_merged = pd.merge(df1_renamed, df2_renamed, on='Номенклатура', how='outer')

    # Функция для сравнения значений по строке
    def get_differences(row):
        diffs = []
        for col in common_columns:
            val1 = row.get(f'{name1}_{col}', None)
            val2 = row.get(f'{name2}_{col}', None)
            if pd.isna(val1) or pd.isna(val2):
                continue
            else:
                tokens1 = tokenize(str(val1))
                tokens2 = tokenize(str(val2))
                if not are_words_possibly_synonyms(tokens1, tokens2, are_synonyms):
                    diffs.append(col)
        return ', '.join(diffs)

    # Добавление столбца с различиями
    df_merged['diff_columns'] = df_merged.apply(get_differences, axis=1)

    return df_merged

# --------------------------------------------------------------Сохранить результат сравнения в excel--------------------------------------------------------------

def save_comparison_to_excel(df: pd.DataFrame, filename: str):
    red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
    yellow_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")

    wb = Workbook()
    ws = wb.active

    # Записываем DataFrame в Excel
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

    headers = [cell.value for cell in ws[1]]
    diff_col_index = len(headers)
    prefix_columns = [col for col in headers if col != 'Номенклатура' and col != 'diff_columns']

    # Автонастройка ширины столбцов по первой строке
    for col_idx, cell in enumerate(ws[1], start=1):
        max_length = len(str(cell.value)) if cell.value else 0
        col_letter = cell.column_letter
        ws.column_dimensions[col_letter].width = max_length + 2  # +2 для отступа

    # Применение стилей и переносов
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
        diff_text = row[diff_col_index - 1].value
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center')  # включаем перенос текста

        if isinstance(diff_text, str) and diff_text.strip():
            different_fields = [field.strip() for field in diff_text.split(',')]
            for diff_field in different_fields:
                for col_idx, col_name in enumerate(headers):
                    if col_name.endswith(f"_{diff_field}"):
                        row[col_idx].fill = yellow_fill
        row[diff_col_index - 1].fill = red_fill

    wb.save(filename)

In [23]:
# Протестировать сравнение двух DataFrame и сохранение результата в excel
from argparse import Namespace

args = Namespace(site1='korting', site2='housedorf')
if not args.site2:
    raise ValueError("there's not enough arguments")
if args.site1 == args.site2:
    raise ValueError("the arguments coincide, comparison is impossible")

resultdf_1 = pd.read_parquet(f"{args.site1}_auxiliary.parquet")
resultdf_2 = pd.read_parquet(f"{args.site2}_auxiliary.parquet")
comp_result = compare_dataframes(resultdf_1, resultdf_2, args.site1, args.site2)

save_comparison_to_excel(comp_result, f'comparison_{args.site1}_vs_{args.site2}.xlsx')

# Программа автоматической проверки обновлений

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import argparse
from itertools import product
from ruwordnet import RuWordNet
import pymorphy2
import re

# ---------------------------------------------------------Добавить функции для сопоставления синонимов---------------------------------------------------------

wordnet = RuWordNet()
morph = pymorphy2.MorphAnalyzer()

def get_normal_form(word):
    return morph.parse(word)[0].normal_form

def are_synonyms(word1, word2):
    lemma1 = get_normal_form(word1)
    lemma2 = get_normal_form(word2)

    synsets1 = wordnet.get_synsets(lemma1)
    synsets2 = wordnet.get_synsets(lemma2)

    # Сравниваем наличие общих лемм в синсетах
    for s1 in synsets1:
        for s2 in synsets2:
            if s1.id == s2.id:
                return True
    return False

def tokenize(text):
    return set(re.findall(r'\w+', text.lower()))

def are_words_possibly_synonyms(words1, words2, are_synonims_func):
    pairs = product(words1, words2)
    for w1, w2 in pairs:
        if w1 == w2:
            return True
        if are_synonims_func(w1, w2):
            return True
    return False

# -------------------------------------------------------------------Сравнить записанные данные-------------------------------------------------------------------

def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame, name1: str = 'df1', name2: str = 'df2') -> pd.DataFrame:
    """
    Сравнивает два DataFrame по общим столбцам, включая "Номенклатура" как ключ.
    Добавляет префиксы к столбцам (кроме "Номенклатура") и формирует колонку diff_columns.
    """
    if 'Номенклатура' not in df1.columns or 'Номенклатура' not in df2.columns:
        raise ValueError("Оба DataFrame должны содержать колонку 'Номенклатура'")

    # Определим общие характеристики (кроме "Номенклатура")
    common_columns = df1.columns.intersection(df2.columns).difference(['Номенклатура'])

    # Сузим входные DataFrame'ы до нужных колонок
    df1_reduced = df1[['Номенклатура'] + list(common_columns)].copy()
    df2_reduced = df2[['Номенклатура'] + list(common_columns)].copy()

    # Переименуем характеристики с префиксами, "Номенклатура" оставим без изменений
    df1_renamed = df1_reduced.rename(columns={col: f'{name1}_{col}' for col in common_columns})
    df2_renamed = df2_reduced.rename(columns={col: f'{name2}_{col}' for col in common_columns})

    # Объединение по "Номенклатура"
    df_merged = pd.merge(df1_renamed, df2_renamed, on='Номенклатура', how='outer')

    # Функция для сравнения значений по строке
    def get_differences(row):
        diffs = []
        for col in common_columns:
            val1 = row.get(f'{name1}_{col}', None)
            val2 = row.get(f'{name2}_{col}', None)
            if pd.isna(val1) or pd.isna(val2):
                continue
            else:
                tokens1 = tokenize(str(val1))
                tokens2 = tokenize(str(val2))
                if not are_words_possibly_synonyms(tokens1, tokens2, are_synonyms):
                    diffs.append(col)
        return ', '.join(diffs)

    # Добавление столбца с различиями
    df_merged['diff_columns'] = df_merged.apply(get_differences, axis=1)

    return df_merged

# --------------------------------------------------------------Сохранить результат сравнения в excel--------------------------------------------------------------

def save_comparison_to_excel(df: pd.DataFrame, filename: str):
    red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
    yellow_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")

    wb = Workbook()
    ws = wb.active

    # Записываем DataFrame в Excel
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

    headers = [cell.value for cell in ws[1]]
    diff_col_index = len(headers)
    prefix_columns = [col for col in headers if col != 'Номенклатура' and col != 'diff_columns']

    # Автонастройка ширины столбцов по первой строке
    for col_idx, cell in enumerate(ws[1], start=1):
        max_length = len(str(cell.value)) if cell.value else 0
        col_letter = cell.column_letter
        ws.column_dimensions[col_letter].width = max_length + 2  # +2 для отступа

    # Применение стилей и переносов
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
        diff_text = row[diff_col_index - 1].value
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center')  # включаем перенос текста

        if isinstance(diff_text, str) and diff_text.strip():
            different_fields = [field.strip() for field in diff_text.split(',')]
            for diff_field in different_fields:
                for col_idx, col_name in enumerate(headers):
                    if col_name.endswith(f"_{diff_field}"):
                        row[col_idx].fill = yellow_fill
        row[diff_col_index - 1].fill = red_fill

    wb.save(filename)

# Протестировать сравнение двух DataFrame и сохранение результата в excel
from argparse import Namespace

args = Namespace(site1='korting', site2='housedorf')
if not args.site2:
    raise ValueError("there's not enough arguments")
if args.site1 == args.site2:
    raise ValueError("the arguments coincide, comparison is impossible")

resultdf_1 = pd.read_parquet(f"{args.site1}_auxiliary.parquet")
resultdf_2 = pd.read_parquet(f"{args.site2}_auxiliary.parquet")
comp_result = compare_dataframes(resultdf_1, resultdf_2, args.site1, args.site2)

save_comparison_to_excel(comp_result, f'comparison_{args.site1}_vs_{args.site2}.xlsx')

# Обработка данных для rag-агента

In [9]:
import requests
from bs4 import BeautifulSoup
import re


def extract_clean_text(url: str, timeout: int = 10) -> str:
    """
    Загружает страницу и возвращает очищенный текст без HTML.
    Подходит для карточек товаров, статей, лендингов.
    """

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/122.0.0.0 Safari/537.36"
        ),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "Referer": "https://www.google.com/",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
    }

    response = requests.get(url, headers=headers, timeout=timeout)
    response.raise_for_status()

    html = response.text

    soup_full = BeautifulSoup(html, "lxml")
    soup = soup_full.body or soup_full

    # 2. Удаляем мусор
    for tag in soup(["script", "style", "noscript", "svg", "img", "iframe"]):
        tag.decompose()

    # 3. Получаем текст
    text = soup.get_text(separator="\n")

    # 4. Нормализация
    text = re.sub(r"\n{3,}", "\n\n", text)   # лишние переводы строк
    text = re.sub(r"[ \t]+", " ", text)      # лишние пробелы
    text = text.strip()

    return text

url = "https://www.mvideo.ru/products/vstraivaemaya-elektricheskaya-plita-evelux-hev-641-b-400008320/reviews?utm_source=google&utm_medium=organic&utm_campaign=google&utm_referrer=google"
text = extract_clean_text(url)

with open("hev_642_b_raw.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Текст сохранён в hev_642_b_raw.txt")


Текст сохранён в hev_642_b_raw.txt


In [15]:
import asyncio
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import re
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright


async def extract_text_playwright(url: str) -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page(locale="ru-RU")

        await page.goto(url, timeout=30000)
        await page.wait_for_load_state("networkidle")

        html = await page.content()
        await browser.close()

    soup = BeautifulSoup(html, "lxml")

    for tag in soup(["script", "style", "noscript", "svg", "img", "iframe"]):
        tag.decompose()

    text = soup.get_text(separator="\n")
    text = re.sub(r"\n{3,}", "\n\n", text).strip()

    return text

url = "https://www.mvideo.ru/products/vstraivaemaya-elektricheskaya-plita-evelux-hev-641-b-400008320/reviews?utm_source=google&utm_medium=organic&utm_campaign=google&utm_referrer=google"
text = await extract_text_playwright(url)

with open("hev_642_b_raw.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Текст сохранён в hev_642_b_raw.txt")

NotImplementedError: 

In [16]:
playwright install

SyntaxError: invalid syntax (938024582.py, line 1)

# Программа processing_similar.py

In [14]:
import pandas as pd
import openpyxl
import re
from itertools import combinations
from collections import defaultdict
from typing import Literal

In [15]:
# Составить Пары совпадающих по названиям столбцов в файле result.xlsx

def find_duplicate_column_pairs(file_path: str):
    """
    Находит пары совпадающих по названию столбцов в Excel-файле.
    
    :param file_path: путь к файлу Excel
    :return: список пар (col_name, index1, index2)
    """
    df = pd.read_excel(file_path, header=0)
    col_names = list(df.columns)
    print(col_names)

    # убираем '.1', '.2', ...
    norm_cols = []
    for c in col_names:
        s = str(c).strip()
        s = re.sub(r"\.\d+$", "", s)  # убираем '.1', '.2', ...
        norm_cols.append(s)

    pairs = []
    for col in set(norm_cols):
        # если имя встречается более одного раза
        indices = [i for i, name in enumerate(norm_cols) if name == col]
        if len(indices) > 1:
            # составляем все возможные пары
            for i1, i2 in combinations(indices, 2):
                pairs.append((col, i1, i2))
    
    return pairs



In [16]:
# Протестировать составление совпадений
pairs = find_duplicate_column_pairs("result.xlsx")
for p in pairs:
    print(p)

['Номенклатура', '^Циф. Арт.', '^Бук. Арт.', '*ЦЕННИК МЕБЕЛЬЩИК', 'EASY INSTALLATION', 'АВТОМАТИКА ЗАКИПАНИЯ', 'АВТОМАТИКА ЗАКИПАНИЯ.1', 'АВТОМАТИЧЕСКИЙ РЕЖИМ', 'АВТОМАТИЧЕСКИЙ РЕЖИМ.1', 'АВТОМАТИЧЕСКИЙ РЕЖИМ (СКОРОСТЬ ВСАСЫВАНИЯ АВТОМАТИЧЕСКИ УСТАНАВЛИВАЕТСЯ ПРИ ПРИГОТОВЛЕНИИ ПИЩИ)', 'АВТОМАТИЧЕСКОЕ ОПРЕДЕЛЕНИЕ ДИАМЕТРА ПОСУДЫ', 'АВТОМАТИЧЕСКОЕ ОПРЕДЕЛЕНИЕ НАЛИЧИЯ ПОСУДЫ', 'АВТОМАТИЧЕСКОЕ РАСПОЗНАВАНИЕ ПОСУДЫ', 'АВТООТКРЫВАНИЕ ДВЕРЦЫ', 'АСПИРАЦИЯ ПО ПЕРИМЕТРУ', 'БЕЗОПАСНОЕ ВЫКЛЮЧЕНИЕ', 'БЕСШУМНАЯ РАБОТА', 'БЛОКИРОВКА КНОПОК', 'БЛОКИРОВКА ОТ ДЕТЕЙ', 'БЛОКИРОВКА ПАНЕЛИ УПРАВЛЕНИЯ (ЗАЩИТА ОТ ДЕТЕЙ)', 'БЛОКИРОВКА УПРАВЛЕНИЯ/ЗАЩИТА ОТ ДЕТЕЙ', 'БРЕНД', 'БЫСТРЫЙ НАГРЕВ ДУХОВКИ', 'БЫСТРЫЙ ПРЕДВАРИТЕЛЬНЫЙ РАЗОГРЕВ', 'варочная', 'ВЕС (КГ)', 'ВЕС БЕЗ УПАКОВКИ, КГ', 'ВЕС БРУТТО (КГ)', 'ВЕС НЕТТО (КГ)', 'ВЕС С УПАКОВКОЙ, КГ', 'ВИД', 'ВИД.1', 'ВИД ДУХОВОГО ШКАФА', 'ВИДЫ НАГРЕВА', 'ВМЕСТИМОСТЬ (БУТЫЛКИ 0.75 Л)', 'ВНУТРЕННЕЕ ОСВЕЩЕНИЕ', 'ВОЗМОЖНОСТЬ ПРОГРАММИРОВАНИЯ', 'ВОЗМОЖНОСТЬ УСТАНОВКИ АКТИВНОГО

In [10]:
# Составить Пары столбцов с _ и без, совпадающих по тексту в верхнем регистре
def find_prefix_column_pairs(file_path: str):
    """
    Находит пары столбцов, совпадающих по префиксу до '_'.
    В паре всегда один столбец без '_' и один с '_'.
    
    :param file_path: путь к Excel файлу
    :return: список пар (base_col, extended_col)
    """
    df = pd.read_excel(file_path, header=0)
    col_names = list(df.columns)

    base_cols = [c for c in col_names if "_" not in c]
    extended_cols = [c for c in col_names if "_" in c]

    pairs = []
    for ext in extended_cols:
        prefix = ext.split("_", 1)[0]  # часть до _
        if prefix in base_cols:
            pairs.append((prefix, ext))
    
    return pairs

In [11]:
# Протестировать составление аналогий
pairs = find_prefix_column_pairs("result.xlsx")
for p in pairs:
    print(p)

('ГЛУБИНА (СМ)', 'ГЛУБИНА (СМ)_Глубина')
('ВЕС БРУТТО (КГ)', 'ВЕС БРУТТО (КГ)_Вес брутто')
('ПОЛЕЗНЫЙ ОБЪЕМ (Л)', 'ПОЛЕЗНЫЙ ОБЪЕМ (Л)_Объем камеры')


In [7]:
# Написать вспомогательную функцию, которая будет из строки делать регулярное выражение

def make_regex_from_string(s: str, spaces=True) -> str:
    # убираем пробелы полностью — чтобы они не влияли на разбиение (или нет)
    if not spaces:
        s = s.replace(" ", "")

    # Разбиваем строку на числа и разделители
    tokens = re.split(r'(\d+(?:\.\d+)?)', s)
    
    parts = []
    for token in tokens:
        if not token: 
            continue
        if re.fullmatch(r'\d+(?:\.\d+)?', token):
            # число: целая часть + опциональная дробная (ровно 1 цифра)
            parts.append(r'\d+(?:\.\d)?')
        elif token.isspace():
            # пробелы -> \s*
            parts.append(r'\s*')
        else:
            # все остальные символы берём как есть (экранируем)
            # пробелы внутри них отдельно не обрабатываем
            parts.append(re.escape(token).replace(r'\ ', r'\s*'))
    
    # Собираем всё и добавляем якоря
    return '^' + ''.join(parts) + '$'

examples = ["2.3 x 59.9 x 45", "24.5x59x60.5", "240x320", "240x320 см"]
i = 0
for ex in examples:
    if i>0 and make_regex_from_string(ex, False) == reg_ex:
        print("True")
    else:
        print("False")
    reg_ex = make_regex_from_string(ex, False)
    print(ex, "->", reg_ex)
    i += 1


False
2.3 x 59.9 x 45 -> ^\d+(?:\.\d)?x\d+(?:\.\d)?x\d+(?:\.\d)?$
True
24.5x59x60.5 -> ^\d+(?:\.\d)?x\d+(?:\.\d)?x\d+(?:\.\d)?$
False
240x320 -> ^\d+(?:\.\d)?x\d+(?:\.\d)?$
False
240x320 см -> ^\d+(?:\.\d)?x\d+(?:\.\d)?см$


In [ ]:
# Написать функцию для подкраски и выявления преобладающего формата
def highlight_dominant_format(file_path: str, pair, flag: Literal["similar", "analogic"] = "similar"):
    """
    Выявляет и подкрашивает преобладающий формат ячеек в Excel-файле.
    
    :param file_path: путь к файлу Excel
    :param flag: тип анализа ("similar" или "analogic")
    """
    df = pd.read_excel(file_path, header=0)
    # Выявить преобладающий формат ячеек в паре. Формат - это регулярное выражение, в котором все цифры заменены на ?
    format_regex = r"\D*?\d+.*?"
    col1, col2 = pair
    series1 = df[col1]
    series2 = df[col2]
    # Применить регулярное выражение к каждой ячейке
    formatted1 = series1.astype(str).replace(format_regex, "?", regex=True)
    formatted2 = series2.astype(str).replace(format_regex, "?", regex=True)
    # Найти преобладающий формат
    dominant_format = formatted1.value_counts().idxmax()
    # Подсветить ячейки в Excel
    highlight_cells(file_path, col1, col2, dominant_format)


In [17]:
def to_regex(s: str) -> str:
    # заменяем все цифры на \d
    return re.sub(r"\d", r"\\d", s)

print(to_regex("1.23 см"))  # \d.\d\d см


\d.\d\d см
